In [1]:
!python -m pip install --upgrade pip
!python -m pip install autogluon

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 17.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split

# 데이터 불러오기
data = pd.read_csv('data_main.csv')

# 클래스 비율 유지해서 train/test 분할
train_df, test_df = train_test_split(
    data,
    test_size=0.2,            # 20%를 test로
    random_state=42,          # 랜덤 시드 고정 (재현성)
    stratify=data['churn']    # stratify 옵션 꼭 넣기!
)

print('Train 클래스 분포:')
print(train_df['churn'].value_counts())
print('\nTest 클래스 분포:')
print(test_df['churn'].value_counts())

Train 클래스 분포:
churn
0    11600
1     1276
Name: count, dtype: int64

Test 클래스 분포:
churn
0    2901
1     319
Name: count, dtype: int64


In [6]:
# AutoGluon 학습 (10분 제한, best quality preset)
predictor = TabularPredictor(label='churn').fit(
    train_df,
    presets='best_quality',
    time_limit=600  # 초 단위 (10분, 필요시 조절)
)

No path specified. Models will be saved in: "AutogluonModels/ag-20250710_053817"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.11.13
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Mar 30 16:01:29 UTC 2025
CPU Count:          2
Memory Avail:       11.37 GB / 12.67 GB (89.7%)
Disk Space Avail:   65.00 GB / 107.72 GB (60.3%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be

(_ray_fit pid=3430) [1000]	valid_set's binary_error: 0.092942
(_ray_fit pid=3620) [1000]	valid_set's binary_error: 0.0880503


(_dystack pid=3212) 	0.9095	 = Validation score   (accuracy)
(_dystack pid=3212) 	61.8s	 = Training   runtime
(_dystack pid=3212) 	1.45s	 = Validation runtime
(_dystack pid=3212) Fitting model: LightGBM_BAG_L1 ... Training model for up to 17.68s of the 64.72s of remaining time.
(_dystack pid=3212) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (2 workers, per: cpus=1, gpus=0, memory=0.36%)
(_ray_fit pid=3974) 	Ran out of time, early stopping on iteration 358. Best iteration is:
(_ray_fit pid=3974) 	[326]	valid_set's binary_error: 0.0901468
(_ray_fit pid=4092) 	Ran out of time, early stopping on iteration 255. Best iteration is: [repeated 2x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)
(_ray_fit pid=4092) 	[213]	valid_set's binary_error: 0.0936408 [repeated 2x across 

In [15]:
# test set에서 예측 및 성능 평가
performance = predictor.evaluate(test_df)
print('\n[전체 모델 성능 평가]')
print(performance)

# 모델별 상세 비교표(Leaderboard)
leaderboard_df = predictor.leaderboard(test_df, silent=True)
display(leaderboard_df)


[전체 모델 성능 평가]
{'accuracy': 0.9127329192546584, 'balanced_accuracy': np.float64(0.5860669599392275), 'mcc': np.float64(0.33875647091330474), 'roc_auc': np.float64(0.7405018699637678), 'f1': 0.28860759493670884, 'precision': 0.75, 'recall': 0.1786833855799373}


,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBMXT_BAG_L2,0.912733,0.914259,accuracy,6.324139,9.496785,308.270509,0.823285,0.423861,55.504223,2,True,10
1,WeightedEnsemble_L3,0.912733,0.914259,accuracy,6.326182,9.498325,308.521478,0.002043,0.001540,0.250969,3,True,13
2,RandomForestGini_BAG_L2,0.912112,0.913016,accuracy,5.726966,9.835511,272.938436,0.226112,0.762586,20.172149,2,True,12
3,LightGBM_BAG_L2,0.911491,0.912783,accuracy,5.909014,9.333576,305.688191,0.408160,0.260652,52.921905,2,True,11
4,LightGBM_BAG_L1,0.910870,0.911230,accuracy,1.956125,1.157668,62.116336,1.956125,1.157668,62.116336,1,True,4
5,WeightedEnsemble_L2,0.910870,0.911230,accuracy,1.959169,1.159445,62.304240,0.003044,0.001777,0.187904,2,True,9
6,RandomForestGini_BAG_L1,0.907764,0.906415,accuracy,0.382253,0.756367,15.801670,0.382253,0.756367,15.801670,1,True,5
7,LightGBMXT_BAG_L1,0.907764,0.908978,accuracy,1.415477,1.025805,54.881157,1.415477,1.025805,54.881157,1,True,3
8,RandomForestEntr_BAG_L1,0.907143,0.907036,accuracy,0.378129,0.727423,15.545791,0.378129,0.727423,15.545791,1,True,6
9,ExtraTreesGini_BAG_L1,0.905590,0.906027,accuracy,0.500469,0.963588,6.657340,0.500469,0.963588,6.657340,1,True,8


In [14]:
from sklearn.metrics import classification_report

# 1. 여러 모델 성능 비교 (표)
leaderboard_df = predictor.leaderboard(test_df, silent=True)
from IPython.display import display
display(leaderboard_df)

# 2. 가장 성능 좋은 모델의 이름 구하기
best_model_name = leaderboard_df.iloc[0]['model']
print(f'\n가장 성능 좋은 모델: {best_model_name}')

# 3. 해당 모델로 예측값 구하기
y_true = test_df['churn']
X_test = test_df.drop(columns=['churn'])
y_pred = predictor.predict(X_test, model=best_model_name)

# 4. precision, recall, F1-score 등 출력
print('\n가장 좋은 모델의 classification report:')
print(classification_report(y_true, y_pred, digits=4))

# =============================
# 5. 베스트 모델의 주요 전처리 옵션(최종 사용 feature) 확인
best_model_obj = predictor._trainer.load_model(best_model_name)
print("\n===== 최종 사용된 Feature List (주요 전처리 결과) =====")
print(best_model_obj.features)

# 6. 베스트 모델의 전체 파이프라인 구조(모델 파라미터 요약)
print("\n===== 베스트 모델 전체 구조 및 파라미터 요약 =====")
# LightGBM/XGBoost/CatBoost 등일 때 파라미터 요약
try:
    # 1) 모델 래퍼 자체 repr()
    print(best_model_obj)
    # 2) 핵심 하위 모델 파라미터 (LightGBM 등)
    if hasattr(best_model_obj, 'model'):
        print("\n[내부 native 모델 repr()]")
        print(best_model_obj.model)
    if hasattr(best_model_obj, 'params'):
        print("\n[AutoGluon이 지정한 하이퍼파라미터]")
        print(best_model_obj.params)
except Exception as e:
    print("모델 구조를 요약하는 데 문제가 있습니다:", e)


,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBMXT_BAG_L2,0.912733,0.914259,accuracy,10.593944,9.496785,308.270509,1.799942,0.423861,55.504223,2,True,10
1,WeightedEnsemble_L3,0.912733,0.914259,accuracy,10.596784,9.498325,308.521478,0.002840,0.001540,0.250969,3,True,13
2,RandomForestGini_BAG_L2,0.912112,0.913016,accuracy,9.274170,9.835511,272.938436,0.480168,0.762586,20.172149,2,True,12
3,LightGBM_BAG_L2,0.911491,0.912783,accuracy,9.681853,9.333576,305.688191,0.887851,0.260652,52.921905,2,True,11
4,LightGBM_BAG_L1,0.910870,0.911230,accuracy,2.935766,1.157668,62.116336,2.935766,1.157668,62.116336,1,True,4
5,WeightedEnsemble_L2,0.910870,0.911230,accuracy,2.938881,1.159445,62.304240,0.003116,0.001777,0.187904,2,True,9
6,RandomForestGini_BAG_L1,0.907764,0.906415,accuracy,0.553825,0.756367,15.801670,0.553825,0.756367,15.801670,1,True,5
7,LightGBMXT_BAG_L1,0.907764,0.908978,accuracy,2.640384,1.025805,54.881157,2.640384,1.025805,54.881157,1,True,3
8,RandomForestEntr_BAG_L1,0.907143,0.907036,accuracy,0.423025,0.727423,15.545791,0.423025,0.727423,15.545791,1,True,6
9,ExtraTreesGini_BAG_L1,0.905590,0.906027,accuracy,0.613960,0.963588,6.657340,0.613960,0.963588,6.657340,1,True,8



가장 성능 좋은 모델: LightGBMXT_BAG_L2

가장 좋은 모델의 classification report:
              precision    recall  f1-score   support

           0     0.9167    0.9935    0.9535      2901
           1     0.7500    0.1787    0.2886       319

    accuracy                         0.9127      3220
   macro avg     0.8333    0.5861    0.6211      3220
weighted avg     0.9002    0.9127    0.8876      3220


===== 최종 사용된 Feature List (주요 전처리 결과) =====
['KNeighborsUnif_BAG_L1', 'KNeighborsDist_BAG_L1', 'LightGBMXT_BAG_L1', 'LightGBM_BAG_L1', 'RandomForestGini_BAG_L1', 'RandomForestEntr_BAG_L1', 'CatBoost_BAG_L1', 'ExtraTreesGini_BAG_L1', 'cons_12m', 'cons_gas_12m', 'cons_last_month', 'forecast_base_bill_ele', 'forecast_bill_12m', 'forecast_cons', 'forecast_cons_12m', 'forecast_cons_year', 'forecast_discount_energy', 'forecast_meter_rent_12m', 'forecast_price_energy_p1', 'forecast_price_energy_p2', 'forecast_price_pow_p1', 'has_gas', 'imp_cons', 'margin_gross_pow_ele', 'margin_net_pow_ele', 'nb_prod_act',